In [1]:
#import ultralytics
#ultralytics.checks()
#from ultralytics import YOLO

#import dayolo
#from dayolo import YOLO

import yolo
from yolo import YOLO

from PIL import Image
import torch
import yaml

import os
import random
#from ultralytics.utils.plotting import plot_labels
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

#import visualization_utils as visutils

/vast/palmer/home.grace/eec42/BirdDetector/src
/vast/palmer/home.grace/eec42/BirdDetector/src/data_preprocessing


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
#pretrained_model_name = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
#model_path = 'runs/detect/' + pretrained_model_name + '/weights/best.pt'
#model = YOLO('yolov8m.pt', task='da_detect')
#model = YOLO("yolov8.yaml").load("yolov8m.pt")
#model = YOLO('yolov8m.pt', task='da_detect')
#model = YOLO('yolov8m.pt', task='detect')
#model = YOLO("yolov8m.yaml", task='detect').load("yolov8m.pt")
model = YOLO("yolov8m.yaml", task='detect', subtask='multifeaturesDC').load("yolov8m.pt")


#model.load_weights('yolov8m.pt')


#PRETRAINED_MODEL_NAME = 'pfeifer_penguins_poland_10percentbckgd_yolov8m_120epoch'
#PRETRAINED_MODEL_PATH = 'src/model/runs/detect/' + PRETRAINED_MODEL_NAME + '/weights/best.pt'

#MODEL_NAME = 'pfeifer_penguins_poland_palmyra_10percent_bckgd_yolov8m_120epochs'
#TASK = 'detect' # Choose between: 'deepcoral_detect' 'detect'
#model = YOLO('yolov8m.pt')

INITIALIZING Multi Features Single CLASSIFIER MODEL

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  yolo.nn.modules.conv.Conv                    [3, 48, 3, 2]                 
  1                  -1  1     41664  yolo.nn.modules.conv.Conv                    [48, 96, 3, 2]                
  2                  -1  2    111360  yolo.nn.modules.block.C2f                    [96, 96, 2, True]             
  3                  -1  1    166272  yolo.nn.modules.conv.Conv                    [96, 192, 3, 2]               
  4                  -1  4    813312  yolo.nn.modules.block.C2f                    [192, 192, 4, True]           
  5                  -1  1    664320  yolo.nn.modules.conv.Conv                    [192, 384, 3, 2]              
  6                  -1  4   3248640  yolo.nn.modules.block.C2f                    [384, 384, 4, True]           
  7                  -1  1   199180

 21                  -1  2   4207104  yolo.nn.modules.block.C2f                    [960, 576, 2]                 
 54        [15, 18, 21]  1   3776275  yolo.nn.modules.head.Detect                  [1, [192, 384, 576]]          
YOLOv8m summary: 295 layers, 25856899 parameters, 25856883 gradients, 79.1 GFLOPs

Transferred 469/475 items from pretrained weights
Transferred 469/475 items from pretrained weights


In [4]:
print(model.task)
print(model.subtask)

detect
multifeaturesDC


In [5]:
model.model.model
for i, a in enumerate(model.model.model):
    if i == 22:
        print("block", i)
        print(a)

block 22
Detect(
  (cv2): ModuleList(
    (0): Sequential(
      (0): Conv(
        (conv): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): Sequential(
      (0): Conv(
        (conv): Conv2d(384, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fal

In [6]:
# TRAIN the model on our dataset (data.yml config file) 

model_name = 'TEST_features_loss'

results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=3,
   batch=8, #32,
   #cos_lr=True,
   #dropout=0.3,
   #optimizer='Adam',
   patience=10,
   device=0,
   verbose=True,
   val = False,
   #lr0=0.001,
   #lrf=0.0001,
   degrees=90, fliplr=0.5, flipud=0.5, scale=0.5, # augmentation parameters
   name=model_name)

New https://pypi.org/project/ultralytics/8.1.9 available 😃 Update with 'pip install -U ultralytics'
New https://pypi.org/project/ultralytics/8.1.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.228 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Ultralytics YOLOv8.0.228 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=data.yaml, epochs=3, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=TEST_features_loss36, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, source_name=global_birds_palmyra, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, m

INITIALIZING Multi Features Single CLASSIFIER MODEL

                   from  n    params  module                                       arguments                     

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  yolo.nn.modules.conv.Conv                    [3, 48, 3, 2]                 
  0                  -1  1      1392  yolo.nn.modules.conv.Conv                    [3, 48, 3, 2]                 
  1                  -1  1     41664  yolo.nn.modules.conv.Conv                    [48, 96, 3, 2]                
  1                  -1  1     41664  yolo.nn.modules.conv.Conv                    [48, 96, 3, 2]                
  2                  -1  2    111360  yolo.nn.modules.block.C2f                    [96, 96, 2, True]             
  2                  -1  2    111360  yolo.nn.modules.block.C2f                    [96, 96, 2, True]             
  3                  -1  1    1662

train: Scanning /gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/train/labels.cache... 1722 images, 182 backgrounds, 0 corrupt: 100%|██████████| 1904/1904 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/val/labels.cache... 248 images, 19 backgrounds, 0 corrupt: 100%|██████████| 267/267 [00:00<?, ?it/s]


...

torch.Size([8, 2])
torch.Size([8, 2])


/home/eec42/.conda/envs/pdm/lib/python3.11/site-packages/torch/autograd/__init__.py:251: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
        3/3      6.82G      2.308      1.932      1.363     0.7437         10        640: 100%|██████████| 238/238 [01:00<00:00,  3.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:09<00:00,  1.73it/s]


                   all        267       1656      0.623      0.576      0.493      0.127
                   all        267       1656      0.623      0.576      0.493      0.127

3 epochs completed in 0.056 hours.

3 epochs completed in 0.056 hours.
Optimizer stripped from runs/detect/TEST_features_loss36/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/TEST_features_loss36/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/TEST_features_loss36/weights/best.pt, 52.0MB
Optimizer stripped from runs/detect/TEST_features_loss36/weights/best.pt, 52.0MB

Validating runs/detect/TEST_features_loss36/weights/best.pt...

Validating runs/detect/TEST_features_loss36/weights/best.pt...
Ultralytics YOLOv8.0.228 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Ultralytics YOLOv8.0.228 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
YOLOv8m summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
YOLOv8m summary (fused): 

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:10<00:00,  1.62it/s]


                   all        267       1656      0.625      0.577      0.494      0.127
                   all        267       1656      0.625      0.577      0.494      0.127
Speed: 0.5ms preprocess, 5.9ms inference, 0.0ms loss, 6.4ms postprocess per image
Speed: 0.5ms preprocess, 5.9ms inference, 0.0ms loss, 6.4ms postprocess per image
Results saved to runs/detect/TEST_features_loss36
Results saved to runs/detect/TEST_features_loss36


In [7]:
batch_idx = torch.Tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 3., 3., 4., 5., 6., 7.])
annotations_mask = [False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False]

new_batch_idx = batch_idx[annotations_mask]
new_batch_idx


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.])

In [8]:
b = new_batch_idx.unique(return_inverse=True)
b[1]


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [9]:

selected_img = []
img_path = '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/test/'
selected_img.extend(random.choices(os.listdir(img_path + '/images/'), k=5))

# Predict results for randomly selected images
results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.1, 
        iou = 0.1,
        show = False,
        save = False
    )



0: 640x640 (no detections), 1: 640x640 102 birds, 2: 640x640 1 bird, 3: 640x640 (no detections), 4: 640x640 (no detections), 129.1ms
0: 640x640 (no detections), 1: 640x640 102 birds, 2: 640x640 1 bird, 3: 640x640 (no detections), 4: 640x640 (no detections), 129.1ms
Speed: 1.8ms preprocess, 25.8ms inference, 11.5ms postprocess per image at shape (1, 3, 640, 640)
Speed: 1.8ms preprocess, 25.8ms inference, 11.5ms postprocess per image at shape (1, 3, 640, 640)


In [10]:
model.model.model
for i, a in enumerate(model.model.model):
    if i == 23:
        print("block", i)
        print(a)

In [11]:
model.subtask

'multifeaturesDC'

In [12]:
print(B)

NameError: name 'B' is not defined

In [ ]:

module_path = os.path.abspath(os.path.join('..', '..'))
print(module_path)
module_path = module_path+'/runs'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

/vast/palmer/home.grace/eec42/BirdDetector
/vast/palmer/home.grace/eec42/BirdDetector/runs


In [ ]:
PRETRAINED_MODEL_PATH = module_path + '/detect/YOLO_pe_palmyra_10percentbkgd_test2/weights/best.pt'
model = YOLO('yolov8m.yaml', task='detect').load(PRETRAINED_MODEL_PATH)


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

In [ ]:
selected_img = []
img_path = '/gpfs/gibbs/project/jetz/eec42/data/pe_palmyra_10percentbkgd/test/'
for subdataset in ['global_birds_penguins', 'global_birds_palmyra']:
    selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=10))

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join(img_path + 'images/', img) for img in selected_img],
        conf = 0.1, 
        iou = 0.1,
        show = False,
        save = False
    )


0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 (no detections), 14: 640x640 (no detections), 15: 640x640 (no detections), 16: 640x640 (no detections), 17: 640x640 (no detections), 18: 640x640 (no detections), 19: 640x640 (no detections), 196.3ms
Speed: 1.5ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
results[5].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (480, 480)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))

## Visualization

In [ ]:
TASK = 'deepcoral_detect' #detect
#model_name = 'deepcoral_test4'
#model_path = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'
#model = YOLO(model_path, TASK)
model_name = 'deepcoral_background_lscale16_epochs20_coralgain10'
PRETRAINED_MODEL_PATH = 'runs/' + TASK + '/' + model_name + '/weights/best.pt'

model = YOLO(PRETRAINED_MODEL_PATH, TASK)

In [ ]:
datasets = ['source', 'target'] #['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins', 'global_birds_pfeifer']
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

In [ ]:
# FOR NORMAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'detect':
    selected_img = []
    for subdataset in datasets:
        selected_img.extend(random.choices(os.listdir(img_path + subdataset + '/images/'), k=6))

    results = model.predict(
            #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
            source = [os.path.join(img_path + 'images/', img) for img in selected_img],
            conf = 0.2, 
            iou = 0.1,
            show=False,
            save=False
        )

In [ ]:
if TASK == 'detect':
    for img, result in zip(selected_img, results):

        detection_boxes = []
        save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
        for detect in range(len(result.boxes.cls)):
            det = {}
            det['conf'] = result.boxes.conf[detect].cpu()
            det['category'] = result.boxes.cls[detect].cpu()
            coords = result.boxes.xywhn[detect].cpu()
            det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
            detection_boxes.append(det)
            
        im_path = os.path.join(img_path + 'images/', img)
        visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                        confidence_threshold=0.0, detector_label_map=None,
                                        thickness=1,expansion=0, colormap=['Red'])

        selected_label = img_path + 'labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
        if os.path.exists(selected_label):
            detection_boxes = []
            df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
            for irow, row in df.iterrows():  
                det = {}
                det['conf'] = None
                det['category'] = row[0]
                det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                detection_boxes.append(det)
        
            # Draw annotations
            save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
            visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['SpringGreen'])
                                            
            # Remove predictions-only images
            os.remove(save_path)


In [ ]:
# FOR DEEP CORAL MODEL
# Select randomly 10 images from the test dataset

if TASK == 'deepcoral_detect':

    for subdataset in datasets:
        selected_img = random.choices(os.listdir(img_path + subdataset + '/images/'), k=12)

        results = model.predict(
                #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
                source = [os.path.join(img_path, subdataset + '/images/', img) for img in selected_img],
                conf = 0.2, 
                iou = 0.1,
                show=False,
                save=False
            )
        
        for img, result in zip(selected_img, results):

            detection_boxes = []
            save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_' + os.path.basename(result.path).split('.jpg')[0] + '.jpg'
            for detect in range(len(result.boxes.cls)):
                det = {}
                det['conf'] = result.boxes.conf[detect].cpu()
                det['category'] = result.boxes.cls[detect].cpu()
                coords = result.boxes.xywhn[detect].cpu()
                det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
                detection_boxes.append(det)
                
            im_path = os.path.join(img_path + subdataset + '/images/', img)
            visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                            confidence_threshold=0.0, detector_label_map=None,
                                            thickness=1,expansion=0, colormap=['Red'])

            selected_label = img_path  + subdataset + '/labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
            if os.path.exists(selected_label):
                detection_boxes = []
                df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
                for irow, row in df.iterrows():  
                    det = {}
                    det['conf'] = None
                    det['category'] = row[0]
                    det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
                    detection_boxes.append(det)
        
                # Draw annotations
                save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/' + TASK + '/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.hpg')[0] + '.jpg'
                visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                                confidence_threshold=0.0, detector_label_map=None,
                                                thickness=1,expansion=0, colormap=['SpringGreen'])
                
                # Remove predictions-only images
                os.remove(save_path)


In [ ]:
b

## YOLO Evaluation

In [ ]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True, iou=0.1, conf=0.2, max_det=600)

# Export the model to ONNX format
#success = model.export(format='onnx')

In [ ]:
print(metrics.box.map)
print(metrics.box.map50)    # map50-95

## YOLO Evaluation per dataset

In [ ]:
datasets =  ['global_birds_poland', 'global_birds_palmyra', 'global_birds_penguins',
                    'global_birds_mckellar', 'global_birds_newmexico', 
                    'global_birds_pfeifer', 'uav_thermal_waterfowl']

In [ ]:
model_name = 'pfeifer_penguins_poland_palmyra_mckellar_yolov8m_120epoch'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
for dataset in datasets:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    metrics = model.val(split='test', save_json=True, iou=0.1, max_det=600)
    print(metrics.box.map50)

# Change test folder to original name
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
data['test'] = 'test/images/'

## **TEST**

In [ ]:
model_name = 'pfeifer_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience50_lr00.001_lrf0.0001'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')

In [ ]:
fname = "data.yaml"
stream = open(fname, 'r')
data = yaml.safe_load(stream)
img_path = data['path'] + '/test/'

selected_img = (random.choices(os.listdir(img_path + '/images/'), k=1))
selected_img = [os.path.join(img_path + '/images/', x) for x in selected_img]
selected_img

In [ ]:
output = model(selected_img)

In [ ]:
output